In [1]:
import pandas as pd

In [2]:
patha =  r'/Users/sanjayshah/Desktop/Tools_Analytics-master/df_processed.csv'
with open(patha):
    df=pd.read_csv(patha)
    

df = df.dropna(how='any',axis=0)

In [4]:
def categorize_data(df):
    
    #This list stores all the categories
    category_list=list()

    
    #Assign categories to each book
    
    for i in range(len(df)):
        if df['Rating'].iloc[i]>='4.5':
            category_list.append(1)
        elif df['Rating'].iloc[i]<'4.5' and df['Rating'].iloc[i]>='4':
            category_list.append(2)
        elif df['Rating'].iloc[i]<'4' and df['Rating'].iloc[i]>='3.5':
            category_list.append(3)
        elif df['Rating'].iloc[i]<'3.5' and df['Rating'].iloc[i]>='3':
            category_list.append(4)
        elif df['Rating'].iloc[i]<'3' and df['Rating'].iloc[i]>='2.5':
            category_list.append(5)
        else:
            category_list.append(0)

    #Add categories to the DF
    df['Category']=category_list
    
    return df

In [5]:
df=categorize_data(df)


In [10]:
for i in [1,2,3,4,5]:
    df=df[df['Category']==i]
    topic_categories(df)

#loop

/Users/sanjayshah/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


original document: 
['', "It's", 'Christmas', 'Eve', 'and', 'the', 'JOLLY', 'POSTMAN', 'is', 'delivering', 'greetings', 'to', 'various', 'fairy-tale', 'characters', '-', "there's", 'a', 'card', 'for', 'Baby', 'Bear,', 'a', 'game', 'appropriately', 'called', "'Beware'", 'for', 'Red', 'Riding', 'Hood', 'from', 'Mr', 'Wolf,', 'a', 'get-well', 'jigsaw', 'for', 'hospitalised', 'Humpty', 'Dumpty', 'and', 'three', 'more', 'surprise', 'envelopes', 'containing', 'letters,', 'cards,', 'etc.', "Everyone's", 'favourite', 'postman', 'keeps', 'on', 'peddling', 'his', 'bicycle', 'up', 'hill', 'and', 'down', 'dale', '.', '.', '.', 'and', 'into', "everybody's", 'hearts.']


 tokenized and lemmatized document: 
['christma', 'jolli', 'postman', 'deliv', 'greet', 'fairi', 'tale', 'charact', 'card', 'babi', 'bear', 'game', 'appropri', 'call', 'bewar', 'rid', 'hood', 'wolf', 'jigsaw', 'hospitalis', 'humpti', 'dumpti', 'surpris', 'envelop', 'contain', 'letter', 'card', 'favourit', 'postman', 'keep', 'peddl',

IndexError: index 0 is out of bounds for axis 0 with size 0

In [7]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
def topic_categories(df):
    

    data_text = df[['Description']]
    data_text['index'] = data_text.index
    documents = data_text

    stemmer = SnowballStemmer('english')
    def lemmatize_stemming(text):
        return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    def preprocess(text):
        result = []
        for token in gensim.utils.simple_preprocess(text):
            if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
                result.append(lemmatize_stemming(token))
        return result

    doc_sample = documents[documents['index'] == 55].values[0][0]
    print('original document: ')
    words = []
    for word in doc_sample.split(' '):
        words.append(word)
    print(words)
    print('\n\n tokenized and lemmatized document: ')
    print(preprocess(doc_sample))
    processed_docs = documents['Description'].map(preprocess)
    dictionary = gensim.corpora.Dictionary(processed_docs)
    count = 0
    for k, v in dictionary.iteritems():
        print(k, v)
        count += 1
        if count > 10:
            break
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    print(bow_corpus[1])
    bow_doc_1 = bow_corpus[1]

    for i in range(len(bow_doc_1)):
        print("Word {} (\"{}\") appears {} time.".format(bow_doc_1[i][0], 
                                                         dictionary[bow_doc_1[i][0]], 
                                                         bow_doc_1[i][1]))
    from gensim import corpora, models

    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    from pprint import pprint

    for doc in corpus_tfidf:
        pprint(doc)
        break
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
    for idx, topic in lda_model.print_topics(-1):
        print('Topic: {} \nWords: {}'.format(idx, topic))
    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
    for idx, topic in lda_model_tfidf.print_topics(-1):
        print('Topic: {} Word: {}'.format(idx, topic))
    print(processed_docs[55])
    for index, score in sorted(lda_model[bow_corpus[1]], key=lambda tup: -1*tup[1]):
        print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))
    for index, score in sorted(lda_model_tfidf[bow_corpus[1]], key=lambda tup: -1*tup[1]):
        print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sanjayshah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
